# Classifying Heart Disease!

### Introduction

Here we introduce the heart disease data set found in the machine learning respository (https://archive.ics.uci.edu/ml/datasets/Heart+Disease). This database contains 76 attributes record for each patient, however, we are only going to focus on a subset of 14 attributes (this is mainly because someone went ahead and clean the data for us down to a subset of 14 kekw. However, I also noticed that attributes of smoking was also part of the datatbase data set. Unfortunately it wasn't included in this 14, but if anyone finds a way to tidy the raw data, including that attribute might be worthwhile to wrangle/classify!). The 14 attributes are as follows:

1. Age
2. Sex
3. Chest pain type (1 = typical angina, 2 = atypical angina, 3 = non-anginal pain, 4 = asymptomatic)
4. Resting blood pressure
5. Serum cholestoral
6. Fasting blood sugar
7. Resting electrocardiographic results (0 = normal, 1 = having ST-T wave abnormality, 2 = left ventricular hypertrophy)
8. Maximum heart rate achieved
9. Exercise induced angina
10. Oldpeak = ST depression induced by exercise relative to rest
11. Slope of the peak exercise ST segment
12. Number of major vessels colored by flourosopy
13. Thalassemia (3 = normal, 6 - fixed defect, 7 = reversable defect)
14. Diagnosis of heart disease (0 = no presence, 1-4 = increasing severity?)

I'm going to attempt to narrow down the attributes we used from this list of 14 by creating scatter plots of Diagnosis of heart disease vs. "other attribute." This should be a good starting point for our proposal.


### Preliminary Exploratory Data Analysis/Method



In [1]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [2]:
#Loading the Data
cleveland_data <- read_delim("processed.cleveland.data", col_names = FALSE)
hungarian_data <- read_delim("processed.hungarian.data", col_names = FALSE)
switzerland_data <- read_delim("processed.switzerland.data", col_names = FALSE)
va_data <- read_delim("processed.va.data", col_names = FALSE)

Rows: 303 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): X12, X13
dbl (12): X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X14

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 294 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (9): X4, X5, X6, X7, X8, X9, X11, X12, X13
dbl (5): X1, X2, X3, X10, X14

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 123 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (9): X4, X6, X7, X8, X9, X10, X11, X12, X13
dbl (5): X1, X2, X3, X5, X14

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or 

In [3]:
#Adding the origin category for each row
cleveland_data <- mutate(cleveland_data, Origin = "cleveland")
hungarian_data <- mutate(hungarian_data, Origin = "hungarian")
switzerland_data <- mutate(switzerland_data, Origin = "switzerland")
va_data <- mutate(va_data, Origin = "Longbeach")

#Creating names for each column
column_names = c("age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "thal", "num", "origin")

#Adding names to columns
colnames(cleveland_data) <- column_names
colnames(hungarian_data) <- column_names
colnames(switzerland_data) <- column_names
colnames(va_data) <- column_names

#Combining the data into a mega data set
heart_data <- rbind(cleveland_data, hungarian_data, switzerland_data, va_data)

glimpse(heart_data)

Rows: 920
Columns: 15
$ age      <dbl> 63, 67, 67, 37, 41, 56, 62, 57, 63, 53, 57, 56, 56, 44, 52, 5…
$ sex      <dbl> 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1…
$ cp       <dbl> 1, 4, 4, 3, 2, 2, 4, 4, 4, 4, 4, 2, 3, 2, 3, 3, 2, 4, 3, 2, 1…
$ trestbps <chr> "145", "160", "120", "130", "130", "120", "140", "120", "130"…
$ chol     <chr> "233", "286", "229", "250", "204", "236", "268", "354", "254"…
$ fbs      <chr> "1", "0", "0", "0", "0", "0", "0", "0", "0", "1", "0", "0", "…
$ restecg  <chr> "2", "2", "2", "0", "2", "0", "2", "0", "2", "2", "0", "2", "…
$ thalach  <chr> "150", "108", "129", "187", "172", "178", "160", "163", "147"…
$ exang    <chr> "0", "1", "1", "0", "0", "0", "0", "1", "0", "1", "0", "0", "…
$ oldpeak  <chr> "2.3", "1.5", "2.6", "3.5", "1.4", "0.8", "3.6", "0.6", "1.4"…
$ slope    <chr> "3", "2", "2", "3", "1", "1", "3", "1", "2", "3", "2", "2", "…
$ ca       <chr> "0.0", "3.0", "2.0", "0.0", "0.0", "0.0", "2.0", "0.0", "1.0"…
$ thal     <chr> "

In [21]:
#Summarizing the presence of heart disease in the patient. 0 is no presence.
total_stats <-  heart_data |> group_by(num) |> count() |> rename("Severity Level of HD" = num, "Total_Count" = n)
c_stats <- cleveland_data |> group_by(num) |> count() |> rename("Severity Level of HD" = num, "C_Count" = n)
h_stats <- hungarian_data |> group_by(num) |> count() |> rename("Severity Level of HD" = num, "H_Count" = n)
s_stats <- switzerland_data |> group_by(num) |> count() |> rename("Severity Level of HD" = num, "S_Count" = n)
v_stats <- va_data |> group_by(num) |> count() |> rename("Severity Level of HD" = num, "V_Count" = n)

#Here we notice that the hungarian data could skew our results as it only classified should we keep the 0-4 values in the diagnosis?
total_stats
c_stats
h_stats
s_stats
v_stats

Severity Level of HD,Total_Count
<dbl>,<int>
0,411
1,265
2,109
3,107
4,28


Severity Level of HD,C_Count
<dbl>,<int>
0,164
1,55
2,36
3,35
4,13


Severity Level of HD,H_Count
<dbl>,<int>
0,188
1,106


Severity Level of HD,S_Count
<dbl>,<int>
0,8
1,48
2,32
3,30
4,5


Severity Level of HD,V_Count
<dbl>,<int>
0,51
1,56
2,41
3,42
4,10


### Expected Outcomes and Significance

